Необходимо написать скрипт для парсинга диалогов из файла `test_data.csv`. 

Главные задачи, которые должен выполнять скрипт:
* Извлекать реплики с приветствием – где менеджер поздоровался. 
* Извлекать реплики, где менеджер представил себя. 
* Извлекать имя менеджера. 
* Извлекать название компании. 
* Извлекать реплики, где менеджер попрощался.
* Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»

Можно создать дополнительное поле в таблице test_data.csv, куда будет сохраняться результат парсинга – например, напротив реплики в столбце “insight” можно ставить флаг того, что эта реплика с приветствием greeting=True

In [1]:
! tree

.
├── Parser.ipynb
└── test_data.csv

0 directories, 2 files


In [2]:
%pip install ipymarkup

Note: you may need to restart the kernel to use updated packages.


In [3]:
import re

import numpy as np
import pandas as pd

from functools import lru_cache
from pymorphy2 import MorphAnalyzer
from razdel import tokenize

pd.set_option('display.max_columns', None)  
pd.set_option('max_colwidth', 800)
pd.set_option('display.expand_frame_repr', False)

In [4]:
df  = pd.read_csv('test_data.csv')

In [104]:
df.head(10)

,dlg_id,line_n,role,text,lemmas,greeting,present
0,0,0,client,Алло,алло,False,None
1,0,1,manager,Алло здравствуйте,алло здравствуйте,True,None
2,0,2,client,Добрый день,добрый день,True,None
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается,я звать ангелина компания диджитал бизнес звонить вы по повод продление лицензия а мы с серый у вы скоро срок заканчиваться,False,"[ангелина, компания, диджитал, бизнес]"
4,0,4,client,Ага,ага,False,None
5,0,5,manager,Угу ну возможно вы рассмотрите и другие варианты видите это хорошая практика сравнивать,угу ну возможно вы рассмотреть и другой вариант видеть это хороший практика сравнивать,False,None
6,0,6,client,Да мы работаем с компанией которая нам подливает поэтому спасибо огромное,да мы работать с компания который мы подливать поэтому спасибо огромный,False,None
7,0,7,client,Как как бы уже до этого момента работаем все устраивает + у нас сопровождение поэтому,как как бы уже до это момент работать всё устраивать + у мы сопровождение поэтому,False,None
8,0,8,manager,Угу а на что вы обращаете внимание при выборе,угу а на что вы обращать внимание при выбор,False,None
9,0,9,client,Как бы нет,как бы нет,False,None


In [6]:
m = MorphAnalyzer()

def words_only(text):
    try:
        return [_.text for _ in list(tokenize(text))]
    except:
        return []
        
@lru_cache(maxsize=2048)
def lemmatize_word(token, pymorphy=m):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text):
    return [lemmatize_word(w) for w in text]

In [7]:
def clean_text(text):
    tokens = words_only(text)
    lemmas = lemmatize_text(tokens)
    
    return ' '.join(lemmas)

In [8]:
from multiprocessing import Pool
from tqdm import tqdm

with Pool(4) as p:
    lemmas = list(tqdm(p.imap(clean_text, df['text']), total=len(df)))
    
df['lemmas'] = lemmas

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 480/480 [00:00<00:00, 7385.10it/s]


`Yargy` lib  

In [9]:
!pip install yargy

In [106]:
from ipymarkup import show_span_ascii_markup as show_markup

from yargy import Parser, rule, and_, not_, or_
from yargy.predicates import dictionary, gram, eq, normalized
from yargy.interpretation import fact, attribute
from yargy.pipelines import morph_pipeline, pipeline
from yargy.parser import prepare_trees
from yargy.tokenizer import MorphTokenizer


In [159]:
Manager = fact(
    'Manager',
    ['greetings', 'name',
    'company', 'goodbuy']
)

### Name

In [165]:
Name = fact(
    'Name',
    ['first']
)

HEADER = rule(
    or_(
    rule(normalized('я')),
    rule(normalized('звать'))
    )
)

FIRST = and_(
    gram('Name'),
    not_(gram('Abbr')),
    not_(gram('PREP'))
).interpretation(Name.first)

NAME = rule(
    HEADER,
    FIRST
).interpretation(Name).interpretation(Manager.name)

### Company

In [171]:
Company = fact(
    'Company',
    ['company_name']
)

COMPANY_NAME = rule(
    eq('компания'),
    gram('ADJF').optional(),
    gram('NOUN').repeatable()
).interpretation(Company.company_name)

COMPANY = rule(
    COMPANY_NAME
).interpretation(Company).interpretation(Manager.company)

### Greetings & Goodbuy

In [172]:
Greetings = fact('Greetings', ['value'])

TYPES = {
    'здравствуйте', 
    'добрый день',
    'добрый'
}

GREETINGS = rule(
    morph_pipeline(TYPES)
).interpretation(Greetings.value).interpretation(Greetings).interpretation(Manager.greetings)

In [173]:
Goodbuy = fact('Goodbuy', ['value'])

TYPES = {
    'до свидания', 
    'всего доброго',
    'хорошего вечера'
}

GOODBUY = rule(
    morph_pipeline(TYPES)
).interpretation(Goodbuy.value).interpretation(Goodbuy).interpretation(Manager.goodbuy)

### Manager parser

In [188]:
MANAGER = or_(
    GREETINGS, 
    NAME,
    COMPANY,
    GOODBUY).interpretation(Manager)

In [189]:
_parser_manager = Parser(MANAGER)

In [215]:
def parser_manager(text):
    matches = list(_parser_manager.findall(text))
    if matches:
        return [match.fact.as_json for match in matches]
    else: return None

In [245]:
## FOR LOCAL DEBUG 

text = 'добрый день я звать ангелина компания диджитал бизнес звонить вы по повод продление а мы сель обратить внимание что '

matches = list(_parser_manager.findall(text))
spans = [_.span for _ in matches]
show_markup(text, spans)

добрый день я звать ангелина компания диджитал бизнес звонить вы по 
───────────   ────────────── ────────────────────────               
повод продление а мы сель обратить внимание что 


In [217]:
with Pool(4) as p:
    lemmas = list(tqdm(p.imap(parser_manager, df['lemmas']), total=len(df)))
    
df['insights'] = lemmas

100%|██████████| 480/480 [00:00<00:00, 2910.10it/s]


In [242]:
df.loc[df['dlg_id']==2].head(4)

,dlg_id,line_n,role,text,lemmas,greeting,present,insights
164,2,0,client,Алло,алло,False,None,None
165,2,1,client,Здравствуйте,здравствуйте,True,None,[{'greetings': {'value': 'здравствуйте'}}]
166,2,2,manager,Алло здравствуйте,алло здравствуйте,True,None,[{'greetings': {'value': 'здравствуйте'}}]
167,2,3,manager,Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там,я звать ангелина компания диджитал бизнес звонить вы по повод продление лицензия а мастер мы с вы сотрудничать по видео там,False,"[ангелина, компания, диджитал, бизнес]","[{'name': {'first': 'ангелина'}}, {'company': {'company_name': 'компания диджитал бизнес'}}]"


In [223]:
df.loc[df['dlg_id']==3].head(3)

,dlg_id,line_n,role,text,lemmas,greeting,present,insights
249,3,0,client,Добрый день,добрый день,True,None,[{'greetings': {'value': 'добрый день'}}]
250,3,1,manager,Алло дмитрий добрый день,алло дмитрий добрый день,True,None,[{'greetings': {'value': 'добрый день'}}]
251,3,2,manager,Добрый меня максим зовут компания китобизнес удобно говорить,добрый я максим звать компания китобизнес удобно говорить,True,None,"[{'greetings': {'value': 'добрый'}}, {'name': {'first': 'максим'}}, {'company': {'company_name': 'компания китобизнес'}}]"


Проверим требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»  

In [244]:
for dlg_id in df['dlg_id'].unique():
    demand = [False, False]
    for ins in df.loc[np.logical_and(
        df['dlg_id']==dlg_id, 
        df['role'] == 'manager')
        ]['insights'].dropna().values:
        
        if 'greetings' in ins[0]:
            demand[0] = True
        if 'goodbuy' in ins[0]:
            demand[1] = True
    if demand == [True, True]:
        print(f'approved for dlg_id: {dlg_id}')
    else:
        print(f'NOT approved for dlg_id: {dlg_id}')

approved for dlg_id: 0
approved for dlg_id: 1
NOT approved for dlg_id: 2
approved for dlg_id: 3
NOT approved for dlg_id: 4
NOT approved for dlg_id: 5
